In [4]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"We're using {device}.")

We're using cuda.


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


分解动作

In [15]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0956, -0.3494,  0.2366,  0.0568,  0.3251, -0.2905, -0.0489,  0.0337,
          0.3948,  0.0531, -0.4958, -0.1254, -0.3286, -0.1359, -0.0480, -0.3968,
          0.5408, -0.2724, -0.0505, -0.4869],
        [ 0.0969, -0.1544, -0.1203, -0.1005,  0.1037, -0.2478, -0.1278,  0.1236,
          0.3304,  0.1494, -0.0701, -0.3262, -0.0124, -0.0220, -0.3591, -0.0699,
          0.2993, -0.4278, -0.0981, -0.5773],
        [ 0.1410, -0.3477, -0.0365,  0.3203,  0.4271, -0.0721, -0.2820,  0.0450,
          0.5010,  0.0334, -0.4720, -0.3304,  0.0723, -0.1592, -0.3823, -0.0790,
          0.3425, -0.4980, -0.0449, -0.3369]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2366, 0.0568, 0.3251, 0.0000, 0.0000, 0.0337, 0.3948,
         0.0531, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5408, 0.0000,
         0.0000, 0.0000],
        [0.0969, 0.0000, 0.0000, 0.0000, 0.1037, 0.0000, 0.0000, 0.1236, 0.3304,
         0.1494, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[-0.0106,  0.1085,  0.2017,  0.0644,  0.0833,  0.1451, -0.2161,  0.1428,
          0.1801,  0.1424],
        [ 0.0701,  0.0437,  0.1253,  0.1080,  0.1257,  0.1675, -0.1480,  0.1139,
          0.1662,  0.1326],
        [ 0.0852,  0.0748,  0.1814,  0.1010,  0.1168,  0.1239, -0.2129,  0.2127,
          0.1565,  0.1337]], grad_fn=<AddmmBackward0>)

In [21]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0904, 0.1018, 0.1118, 0.0974, 0.0993, 0.1056, 0.0736, 0.1054, 0.1094,
         0.1053],
        [0.0976, 0.0951, 0.1032, 0.1014, 0.1032, 0.1076, 0.0785, 0.1020, 0.1075,
         0.1039],
        [0.0982, 0.0972, 0.1082, 0.0998, 0.1014, 0.1021, 0.0729, 0.1116, 0.1055,
         0.1031]], grad_fn=<SoftmaxBackward0>)

In [22]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0296,  0.0143, -0.0182,  ..., -0.0019, -0.0189,  0.0238],
        [ 0.0033,  0.0075, -0.0281,  ..., -0.0002,  0.0067,  0.0213]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0282,  0.0176], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0348,  0.0284, -0.0395,  ...,  0.0321,  0.0121, -0.0398],
        [ 0.0012, -0.0296, -0.0301,  ...,  0.0142, -0.0317,  0.0031]],
       device='cuda:0', grad_fn=<Sl